In [1]:
import subprocess
import json

In [2]:
command = [
    'ffmpeg',
]

try:
    subprocess.run(command, check=True, capture_output=True, text=True)
    print("Conversion terminée avec succès.")
except subprocess.CalledProcessError as e:
    print(f"Erreur lors de la conversion : {e.stderr}")

Erreur lors de la conversion : ffmpeg version 8.0.1-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 17.0.0 (clang-1700.4.4.1)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libharfbuzz --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxml2 --enable-libxvid --enable-libzimg --enable-libzmq --enable-l

In [3]:
def get_audio_info_1(filename):
    command = [
        'ffprobe', 
        '-v', 'quiet', 
        '-print_format', 'json', 
        '-show_format', 
        '-show_streams', 
        filename
    ]

    result = subprocess.run(command, capture_output=True, text=True)
    return json.loads(result.stdout)

In [4]:
get_audio_info_1('input/Album-live-Louez-Yah-YHWH.mp3')

{'streams': [{'index': 0,
   'codec_name': 'mp3',
   'codec_long_name': 'MP3 (MPEG audio layer 3)',
   'codec_type': 'audio',
   'codec_tag_string': '[0][0][0][0]',
   'codec_tag': '0x0000',
   'sample_fmt': 'fltp',
   'sample_rate': '48000',
   'channels': 2,
   'channel_layout': 'stereo',
   'bits_per_sample': 0,
   'initial_padding': 0,
   'r_frame_rate': '0/0',
   'avg_frame_rate': '0/0',
   'time_base': '1/14112000',
   'start_pts': 0,
   'start_time': '0.000000',
   'duration_ts': 4833416448,
   'duration': '342.504000',
   'bit_rate': '128000',
   'disposition': {'default': 0,
    'dub': 0,
    'original': 0,
    'comment': 0,
    'lyrics': 0,
    'karaoke': 0,
    'forced': 0,
    'hearing_impaired': 0,
    'visual_impaired': 0,
    'clean_effects': 0,
    'attached_pic': 0,
    'timed_thumbnails': 0,
    'non_diegetic': 0,
    'captions': 0,
    'descriptions': 0,
    'metadata': 0,
    'dependent': 0,
    'still_image': 0,
    'multilayer': 0}}],
 'format': {'filename': 'inpu

In [5]:
def get_audio_bit_rate_1(filename):
    command = [
        'ffprobe', 
        '-v', 'error', 
        # '-select_streams', 'a',
        '-show_entries', 'stream=bit_rate',
        # '-of', 'default=noprint_wrappers=1:nokey=1',
        '-print_format', 'default=noprint_wrappers=1:nokey=1', 
        filename
    ]

    result = subprocess.run(command, capture_output=True, text=True)
    # return result.stdout
    return result


In [6]:
def get_audio_bit_rate_2(filename):
    command = [
        'ffprobe', 
        '-v', 'error', 
        '-select_streams', 'a',
        '-show_entrie', 'stream=bit_rate',
        '-print_format', 'json',
        # '-print_format', 'default=noprint_wrappers=1:nokey=1',
        filename
    ]

    result = subprocess.run(
        command, 
        stdout=subprocess.PIPE, # The output is stored in the result.stdout variable instead of being printed to the screen.
        stderr=subprocess.STDOUT, # Both your regular data and your error messages are combined into the standard output stream.
    )
    try:
      value = (json.loads(result.stdout)['streams'][0]["bit_rate"])
    except:
      raise Exception(result.stdout)
    return value

In [7]:
def get_audio_bit_rate(filename):
    command = [
        'ffprobe', 
        '-v', 'error', 
        '-select_streams', 'a',
        '-show_entries', 'stream=bit_rate',
        '-print_format', 'json',
        # '-print_format', 'default=noprint_wrappers=1:nokey=1',
        filename
    ]
    value = -1
    try:
        result = subprocess.run(
            command, 
            capture_output=True,       # 2. Capture stdout and stderr
            text=True,                 # 3. Get strings, not bytes
            check=True,                # 4. Raise error if command fails
            timeout=10                 # 5. Prevent hanging forever
        )
        # value = result.stdout
        value = int(json.loads(result.stdout)['streams'][0]["bit_rate"])
    
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error: {e.stderr}")
    
    except subprocess.TimeoutExpired:
        print("The command took too long!")
    
    return value

In [8]:
get_audio_bit_rate_1('input/Album-live-Louez-Yah-YHWH.mp3')

CompletedProcess(args=['ffprobe', '-v', 'error', '-show_entries', 'stream=bit_rate', '-print_format', 'default=noprint_wrappers=1:nokey=1', 'input/Album-live-Louez-Yah-YHWH.mp3'], returncode=0, stdout='128000\n', stderr='')

In [9]:
get_audio_bit_rate_2('input/Album-live-Louez-Yah-YHWH.mp3')

Exception: b"Failed to set value 'stream=bit_rate' for option 'show_entrie': Option not found\n"

In [10]:
get_audio_bit_rate('input/Album-live-Louez-Yah-YHWH.mp3')

128000

## 3. Audio Analysis

In [11]:
def get_audio_info(filename):
    command = [
        'ffprobe', 
        '-v', 'error', 
        '-select_streams', 'a',
        '-show_entries', 'stream=bit_rate,codec_name,sample_rate',
        '-print_format', 'json',
        # '-print_format', 'default=noprint_wrappers=1:nokey=1',
        filename
    ]
    value = -1
    try:
        result = subprocess.run(
            command, 
            capture_output=True,       # 2. Capture stdout and stderr
            text=True,                 # 3. Get strings, not bytes
            check=True,                # 4. Raise error if command fails
            timeout=10                 # 5. Prevent hanging forever
        )
        # value = result.stdout
        value = json.loads(result.stdout)['streams'][0]
        value['bit_rate'] = int(value['bit_rate'])         # Specify the bitrate in bits/s.
        value['sample_rate'] = int(value['sample_rate'])   # Specify the sample rate in Hz. 
                                   
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error: {e.stderr}")
    
    except subprocess.TimeoutExpired:
        print("The command took too long!")
    
    return value

In [12]:
metadata = get_audio_info('input/Album-live-Louez-Yah-YHWH.mp3')
metadata

{'codec_name': 'mp3', 'sample_rate': 48000, 'bit_rate': 128000}

## 4. Audio Quality Profiles

In [13]:
codec_default = "aac"
sample_rate_default = "44100"
channels_default = "2"

In [18]:
input_filename = 'input/Album-live-Louez-Yah-YHWH.mp3'

output_directory = 'output/very_low'
output_filename = f'{output_directory}/playlist.m3u8'
segment_filename = f'{output_directory}/segment_%03d.ts'

In [59]:
profiles = {
    'very_low':  {'bitrate': 64,  'codec': 'aac', 'sample_rate': '44100', 'channels': 2},
    'low':       {'bitrate': 96,  'codec': 'aac', 'sample_rate': '44100', 'channels': 2},
    'normal':    {'bitrate': 128, 'codec': 'aac', 'sample_rate': '44100', 'channels': 2},
    'high':      {'bitrate': 192, 'codec': 'aac', 'sample_rate': '44100', 'channels': 2},
    'very_high': {'bitrate': 256, 'codec': 'aac', 'sample_rate': '44100', 'channels': 2},
}

In [61]:
audio_quality_profiles = [
    {'profile': 'very_low', 'bitrate': 64},
    {'profile': 'low',      'bitrate': 96},
    {'profile': 'normal',   'bitrate': 128},
    {'profile': 'high',     'bitrate': 192},
    {'profile': 'very_high', 'bitrate': 256}
]

In [79]:
AUDIO_QUALITY_PROFILES = {
    'very_low': 64,
    'low': 96,
    'normal': 128,
    'high': 192,
    'very_high': 256
}

In [83]:
list(AUDIO_QUALITY_PROFILES.values())

[64, 96, 128, 192, 256]

In [63]:
audio_quality_profiles

[{'profile': 'very_low', 'bitrate': 64},
 {'profile': 'low', 'bitrate': 96},
 {'profile': 'normal', 'bitrate': 128},
 {'profile': 'high', 'bitrate': 192},
 {'profile': 'very_high', 'bitrate': 256}]

## 5. HLS Generation

○ Fixed segment duration  
○ Independent segments   
○ Deterministic file naming   : OK

In [19]:
def generate_variants(filename, bitrate, codec, sample_rate, channels, segment, output):
    command = [
        'ffmpeg',
        '-i', filename,
        '-ar', sample_rate,
        '-ac:a', channels,
        '-codec:a', codec,
        '-b:a', str(bitrate) + 'k',
        '-hls_segment_filename', segment,
        output
    ]
    try:
        result = subprocess.run(
            command, 
            capture_output=True,       # 2. Capture stdout and stderr
            text=True,                 # 3. Get strings, not bytes
            check=True,                # 4. Raise error if command fails
            timeout=10                 # 5. Prevent hanging forever
        )
        return result
                                   
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error: {e.stderr}")
    
    except subprocess.TimeoutExpired:
        print("The command took too long!")

In [50]:
import os
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

In [21]:
generate_variants(
    filename=input_filename,
    bitrate=64,
    codec=codec_default,
    sample_rate=sample_rate_default,
    channels=channels_default,
    segment=segment_filename,
    output=output_filename
)

CompletedProcess(args=['ffmpeg', '-i', 'input/Album-live-Louez-Yah-YHWH.mp3', '-ar', '44100', '-ac:a', '1', '-codec:a', 'aac', '-b:a', '64k', '-hls_segment_filename', 'output/very_low/segment_%03d.ts', 'output/very_low/playlist.m3u8'], returncode=0, stdout='', stderr="ffmpeg version 8.0.1-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2025 the FFmpeg developers\n  built with Apple clang version 17.0.0 (clang-1700.4.4.1)\n  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libharfbuzz --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame -

In [55]:
def generate_map_variants(filename, codec, sample_rate, channels, segment, output):
    command = [
        'ffmpeg',
        '-i', filename,
        '-ar', sample_rate,
        '-ac:a', channels,
        '-codec:a', codec,
        '-b:a:0', '64k', '-b:a:1', '96k', 
        '-map', '0:a', '-map', '0:a', 
        '-f', 'hls',
        '-var_stream_map', 'a:0,name:very_low a:0,name:very_low',
        '-master_pl_name', 'master.m3u8',
        '-hls_segment_filename', 'output/%v/segment_%03d.ts',
        'output/%v/playlist.m3u8'
    ]
    
    try:
        result = subprocess.run(
            command, 
            capture_output=True,       # 2. Capture stdout and stderr
            text=True,                 # 3. Get strings, not bytes
            check=True,                # 4. Raise error if command fails
            timeout=10                 # 5. Prevent hanging forever
        )
        return result
                                   
    except subprocess.CalledProcessError as e:
        print(f"Command failed with error: {e.stderr}")
    
    except subprocess.TimeoutExpired:
        print("The command took too long!")

In [56]:
generate_map_variants(
    filename=input_filename,
    codec=codec_default,
    sample_rate=sample_rate_default,
    channels=channels_default,
    segment=segment_filename,
    output=output_filename
)

CompletedProcess(args=['ffmpeg', '-i', 'input/Album-live-Louez-Yah-YHWH.mp3', '-ar', '44100', '-ac:a', '1', '-codec:a', 'aac', '-b:a:0', '64k', '-map', '0:a', '-f', 'hls', '-var_stream_map', 'a:0,name:very_low', '-master_pl_name', 'master.m3u8', '-hls_segment_filename', 'output/%v/segment_%03d.ts', 'output/%v/playlist.m3u8'], returncode=0, stdout='', stderr="ffmpeg version 8.0.1-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2025 the FFmpeg developers\n  built with Apple clang version 17.0.0 (clang-1700.4.4.1)\n  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libharfbuzz --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --

In [199]:
metadata = get_audio_info_1('input.mp4')
duration = metadata['format']['duration']
print(f"La durée de la vidéo est de {duration} secondes.")

KeyError: 'format'

In [ ]:
def get_duration(filename):
    result = subprocess.run(["ffprobe", "-ignore_chapters", "1", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return '{:08.4f}'.format(float(result.stdout))


def get_dims(filename):
    result = subprocess.run(["ffprobe", "-ignore_chapters", "1", "-v", "error", "-show_entries",
                             "stream=width,height", "-of",
                             "json", filename],
      stdout=subprocess.PIPE,
      stderr=subprocess.STDOUT)
    #print(json.loads(result.stdout)['streams'])
    try:
      val = (json.loads(result.stdout)['streams'][0]["width"],json.loads(result.stdout)['streams'][0]["height"])
    except:
      val = (json.loads(result.stdout)['streams'][1]["width"],json.loads(result.stdout)['streams'][1]["height"])
    return val


def get_fps(filename):
    result = subprocess.run(["ffprobe", "-ignore_chapters", "1", "-v", "error", "-show_entries",
                             "stream=r_frame_rate", "-of",
                             "json", filename],
      stdout=subprocess.PIPE,
      stderr=subprocess.STDOUT)
    #print(json.loads(result.stdout)['streams'])
    try:
      val = json.loads(result.stdout)['streams'][0]['r_frame_rate']
    except:
      val = json.loads(result.stdout)['streams'][1]['r_frame_rate']
    return '{:02.2f}'.format(eval(val))